In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
import time, os, random, sys
import math
import hyperopt.tpe
import hpsklearn.components
import hpsklearn.demo_support
import warnings

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1500)

warnings.filterwarnings('ignore')
np.random.seed(1)

/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [15]:
def gbdt_lr_train_test(File):
    #GBDT
    start = time.clock()
    train_df, test_df = train_test_split(File, train_size = 0.8)
    X_train = train_df.drop(train_df.columns[0], axis=1)
    y_train = train_df[train_df.columns[0]]
    X_test = test_df.drop(test_df.columns[0], axis=1)
    y_test = test_df[test_df.columns[0]]
    
    gbclf = GradientBoostingClassifier(random_state=42)
    gbclf.fit(X_train,y_train)
    X_train_leaves = gbclf.apply(X_train)[:,:,0]
    (train_rows, cols) = X_train_leaves.shape
    gbclf.fit(X_test,y_test)
    X_test_leaves = gbclf.apply(X_test)[:,:,0]
    
    lr = LogisticRegression(penalty='l1', C=0.1)
    lr.fit(X_train_leaves, y_train)
    y_pred_gbdtlr1 = lr.predict(X_test_leaves)
    gbdtlr_auc1 = roc_auc_score(y_test, y_pred_gbdtlr1)
    print('GBDT+LR auc  : %.5f' % gbdtlr_auc1)
    f_time =time.clock()-start
    print('GBDT+LR time taken: %.2f'% f_time)
    #training xgboost
    start = time.clock()
    xgbclf = xgb.XGBClassifier(random_state=42)
    xgbclf.fit(X_train, y_train)
    
    #using xgboost to encode train set and test set features
    X_train_leaves = xgbclf.apply(X_train)
    train_rows = X_train_leaves.shape[0]
    xgbclf.fit(X_test, y_test)
    X_test_leaves = xgbclf.apply(X_test)
    
    
    #fittting lr using just xgboost encoded feature
    lr = LogisticRegression( C=0.1, penalty='l1')
    lr.fit(X_train_leaves,y_train)
    
    y_pred = lr.predict(X_test_leaves)
    gbdtlr_auc1 = roc_auc_score(y_test, y_pred)
    print('XGBDT+LR AUC : %.5f' % gbdtlr_auc1)
    
    f_time =time.clock()-start
    print('XGBDT+LR time taken: %.2f'% f_time)

In [10]:
example=pd.read_csv('example.csv')

In [11]:
gbdt_lr_train_test(example)      # trial 1

GBDT+LR auc  : 0.56903
GBDT+LR time taken: 11.79
XGBDT+LR AUC : 0.46416
XGBDT+LR time taken: 16.71


In [12]:
gbdt_lr_train_test(example)     # trial 2

GBDT+LR auc  : 0.56998
GBDT+LR time taken: 12.72
XGBDT+LR AUC : 0.47052
XGBDT+LR time taken: 15.83


In [13]:
gbdt_lr_train_test(example)      # trial 3

GBDT+LR auc  : 0.61470
GBDT+LR time taken: 13.07
XGBDT+LR AUC : 0.66579
XGBDT+LR time taken: 10.73


In [14]:
gbdt_lr_train_test(example)       # trial 4

GBDT+LR auc  : 0.71012
GBDT+LR time taken: 11.10
XGBDT+LR AUC : 0.52431
XGBDT+LR time taken: 20.33


In [16]:
gbdt_lr_train_test(example)      # random_state applied for both classifier

GBDT+LR auc  : 0.55783
GBDT+LR time taken: 9.89
XGBDT+LR AUC : 0.56154
XGBDT+LR time taken: 12.90


In [17]:
gbdt_lr_train_test(example)      # random_state applied for both classifier

GBDT+LR auc  : 0.34758
GBDT+LR time taken: 8.49
XGBDT+LR AUC : 0.62272
XGBDT+LR time taken: 10.22


In [18]:
gbdt_lr_train_test(example)      # random_state applied for both classifier

GBDT+LR auc  : 0.36613
GBDT+LR time taken: 5.47
XGBDT+LR AUC : 0.36081
XGBDT+LR time taken: 11.53
